#Nigerian Names Dataset Generator
### Project Description
This is a fun project that allows you to easily generate Nigerian names based of selectin the popular tribes in Nigeria.

### Installations

In [1]:
!pip install -q --upgrade bitsandbytes accelerate gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.8/375.8 kB 18.8 MB/s eta 0:00:00


### Imports

In [2]:
# Imports
import os
import requests
import json
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig
import torch
import gradio as gr
from google.colab import userdata
import gc

### Logging into Huggingface

In [3]:
hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

In [4]:
PHI = "microsoft/Phi-4-mini-instruct"

### System Prompt and User prompt Template

In [18]:
def generate_prompt(tribe, number):
    tribe_patterns = {
        "Igbo": 'names starting with "Chi" (God), "Nneka" (mother is supreme), "Eze" (king), and "Nwa" (child)',
        "Yoruba": 'names starting with "Ade" (crown), "Olu" (God/Lord), "Ayo" (joy), and "Ife" (love)',
        "Hausa": 'names like "Aisha", "Fatima", "Muhammad", and "Ibrahim" reflecting Islamic influence',
    }
    naming_pattern = tribe_patterns.get(
        tribe,
        "meaningful translations and reflect the circumstances of birth, family values, or spiritual beliefs"
    )

    system_prompt = f"""You are a Nigerian name generator specializing in {tribe} names. When asked to generate names, follow these rules:

1. Generate exactly {number} unique first names from the {tribe} tribe of Nigeria
2. Never repeat any name in your list
3. Provide only first names (no surnames or family names)
4. Use authentic {tribe} names with their traditional spellings
5. Include a mix of male and female names unless otherwise specified
6. Present the names in a simple numbered list format
7. After the list, you may optionally provide brief context about {tribe} naming traditions if requested

{tribe} names often have {naming_pattern}.

Ensure all names are culturally authentic and respectful of {tribe} heritage."""

    messages = [

        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"Generate a list of {number} Nigerian {tribe} names"}
    ]

    return messages

In [19]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

In [20]:
def generate_names_interface(tribe, number):

    try:
        messages = generate_prompt(tribe, number)
        tokenizer = AutoTokenizer.from_pretrained(PHI)
        tokenizer.pad_token = tokenizer.eos_token
        input_ids = tokenizer.apply_chat_template(
            messages,
            return_tensors="pt",
            add_generation_prompt=True
        ).to("cuda")

        attention_mask = torch.ones_like(input_ids, dtype=torch.long, device="cuda")
        model = AutoModelForCausalLM.from_pretrained(
            PHI,
            quantization_config=quant_config
        ).to("cuda")
        outputs = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_new_tokens=300,
            do_sample=True,
            temperature=0.7
        )

        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Extract only the assistant's response
        if "<|assistant|>" in generated_text:
            result = generated_text.split("<|assistant|>")[-1].strip()
        else:
            result = generated_text

        del model
        torch.cuda.empty_cache()
        gc.collect()

        return result

    except Exception as e:
        return f"Error generating names: {str(e)}"

In [22]:
def create_interface():
  with gr.Blocks(title="Nigerian Names Generator", theme=gr.themes.Soft()) as demo:
        gr.Markdown("# 🇳🇬 Nigerian Names Dataset Generator")
        gr.Markdown("Generate authentic Nigerian names from the Igbo, Yoruba, or Hausa tribes.")

        with gr.Row():
            with gr.Column():
                tribe_dropdown = gr.Dropdown(
                    choices=["Igbo", "Yoruba", "Hausa"],
                    label="Select Tribe",
                    value="Igbo",
                    info="Choose a Nigerian tribe"
                )

                number_slider = gr.Slider(
                    minimum=1,
                    maximum=20,
                    step=1,
                    value=10,
                    label="Number of Names",
                    info="How many names do you want to generate?"
                )

                generate_btn = gr.Button("Generate Names", variant="primary", size="lg")

            with gr.Column():
                output_text = gr.Textbox(
                    label="Generated Names",
                    lines=15,
                    placeholder="Your generated names will appear here...",
                    show_copy_button=True
                )

        gr.Markdown("""
        ### About
        This tool generates authentic Nigerian names based on traditional naming conventions:
        - **Igbo**: Names often reflect spiritual beliefs (Chi - God, Eze - King)
        - **Yoruba**: Names reflect circumstances of birth (Ade - Crown, Ayo - Joy)
        - **Hausa**: Names often have Islamic influence
        """)

        # Connect the button to the function
        generate_btn.click(
            fn=generate_names_interface,
            inputs=[tribe_dropdown, number_slider],
            outputs=output_text
        )

        return demo

In [23]:
demo = create_interface()
demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a52406050690b5663b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
